Detección de caras con webcam

In [2]:
import cv2
import dlib
from math import hypot
from math import atan2
from math import pi

In [6]:
def rotate_image(image, angle, filter_name):
    image_center = (775,700)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result


filter_name = 'thug.png'
showLandmark = 'true'


cap = cv2.VideoCapture(0)


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")


while True:
    Sucess, img = cap.read()
    img = cv2.flip(img, 1)
    
    imgGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(img)
    for face in faces:
        landmarks = predictor(imgGray, face)
        top_nose = (landmarks.part(29).x, landmarks.part(29).y)
        center_nose = (landmarks.part(30).x, landmarks.part(30).y)
        left_nose = (landmarks.part(31).x, landmarks.part(31).y)
        right_eye = (landmarks.part(45).x, landmarks.part(45).y)
        left_mouth = (landmarks.part(48).x, landmarks.part(48).y)
        right_mouth = (landmarks.part(54).x, landmarks.part(54).y)
        left_eye = (landmarks.part(36).x, landmarks.part(36).y)
        right_nose = (landmarks.part(35).x, landmarks.part(35).y)
        up_mouth = (landmarks.part(51).x, landmarks.part(51).y)
        down_mouth = (landmarks.part(57).x, landmarks.part(57).y)
        down_face = (landmarks.part(8).x, landmarks.part(8).y)
      
        
        front_filter = cv2.imread('./filters/'+filter_name)
        front_filter = cv2.copyMakeBorder(front_filter, 500, 500, 500, 500, cv2.BORDER_CONSTANT, value=[0,0,0,0])
        rotation = -atan2(right_eye[1]-left_eye[1],right_eye[0]-left_eye[0])/pi*180
        front_filter = rotate_image(front_filter, rotation, filter_name)
        
        nose_width = int(hypot(left_eye[0]-right_eye[0], left_eye[1]-right_eye[1])*4.6)
        nose_height = int(nose_width * 0.6708*1)
        offset = int(nose_width*18/200)
        top_left = (int(center_nose[0]-nose_width/2),int(center_nose[1]-nose_height/2)-offset)
        bottom_right = (int(center_nose[0]+nose_width/2),int(center_nose[1]+nose_height/2)-offset)
        
        ffilter = cv2.resize(front_filter, (nose_width, nose_height))
        filter_gray = cv2.cvtColor(ffilter, cv2.COLOR_BGR2GRAY)
        _, filter_mask = cv2.threshold(filter_gray, 0, 255, cv2.THRESH_BINARY_INV)
        
        filter_area = img[max(0,top_left[1]):min(480,top_left[1]+nose_height),max(0,top_left[0]):min(640,top_left[0]+nose_width)]
        filter_mask = filter_mask[max(0,-top_left[1]):max(0,-top_left[1])+filter_area.shape[0],max(0,-top_left[0]):max(0,-top_left[0])+filter_area.shape[1]]
        
        filter_alpha = cv2.bitwise_and(filter_area, filter_area, mask=filter_mask)
        final_nose = cv2.add(filter_alpha, ffilter[max(0,-top_left[1]):filter_area.shape[0]+max(0,-top_left[1]),max(0,-top_left[0]):max(0,-top_left[0])+filter_area.shape[1]])

        img[max(0,top_left[1]):min(480,top_left[1]+nose_height),max(0,top_left[0]):min(640,top_left[0]+nose_width)] = final_nose
    
    if showLandmark == 'true':
        for i in range(68):
            cv2.circle(img, (landmarks.part(i).x, landmarks.part(i).y), radius=0, color=(0, 0, 255), thickness=5)
        
    cv2.imshow("Filtro", img)
    
    tec = cv2.waitKey(40)
    if tec & tec == 27:  # Esc
        break
    
        
cap.release()
cv2.destroyAllWindows()